# Ozone Transformer Example

## Environments

1. Create Python virtual environment named `venv`
   - `python -m venv venv`
2. Activate the new virtual environment `venv`
   - `source venv/bin/activate`
3. Install required python libraries from `ot_requirements.txt` file
   - `pip install -r ot_requirements.txt`
4. Select `venv` as python kernel for this JupyterNotebook

## Ozone Transformer

In [1]:
import torch